In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator as IdealEstimator
from qiskit_aer.primitives import Estimator as NoisyEstimator
from qiskit_aer.noise import NoiseModel, pauli_error
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms.optimizers import SPSA
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Ideal VQE Simulation
# Construct the Hamiltonian for H2
paulis = ["II", "IZ", "ZI", "ZZ", "XX"]
coeffs = [-1.052, 0.398, -0.398, -0.011, 0.181]
h2_hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

# Create the ansatz
ansatz = TwoLocal(2, ['ry', 'rz'], 'cx', reps=1)

# Set up SPSA optimizer
optimizer = SPSA(maxiter=100)

# Use Ideal Estimator for noise-free simulation
ideal_estimator = IdealEstimator()

# Define cost function for ideal case
def ideal_cost(params):
    return ideal_estimator.run(
        circuits=[ansatz], parameter_values=[params], observables=[h2_hamiltonian]
    ).result().values[0]

init_params = np.random.uniform(0, 2 * np.pi, ansatz.num_parameters)
result = optimizer.optimize(ansatz.num_parameters, ideal_cost, initial_point=init_params)

print("\n--- Part 1: Ideal VQE ---")
print("Ground state energy (ideal):", result[1])

# Part 2: VQE on Noisy Simulator
# Define Noise Model
noise_model = NoiseModel()
bit_flip = pauli_error([('X', 0.05), ('I', 0.95)])
noise_model.add_all_qubit_quantum_error(bit_flip, ['cx'])

# Use Noisy Estimator
noisy_estimator = NoisyEstimator(noise_model=noise_model)

# Define cost function for noisy case
def noisy_cost(params):
    return noisy_estimator.run(
        circuits=[ansatz], parameter_values=[params], observables=[h2_hamiltonian]
    ).result().values[0]

result_noisy = optimizer.optimize(ansatz.num_parameters, noisy_cost, initial_point=init_params)

print("\n--- Part 2: Noisy VQE ---")
print("Ground state energy (noisy):", result_noisy[1])

# Part 3: 3-Qubit Bit-Flip Code
# Encoding Circuit
def bit_flip_encode():
    qc = QuantumCircuit(3)
    qc.cx(0, 1)
    qc.cx(0, 2)
    return qc

# Syndrome Measurement Circuit
def syndrome_measure():
    qc = QuantumCircuit(3, 2)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.measure(1, 0)
    qc.measure(2, 1)
    return qc

# Correction Circuit
def correction():
    qc = QuantumCircuit(3, 2)
    qc.x(0).c_if(qc.cregs[0], 1)
    qc.x(2).c_if(qc.cregs[0], 2)
    qc.x(1).c_if(qc.cregs[0], 3)
    return qc

# Part 4: Integrated QEC-VQE
# Create Protected Ansatz
def protected_ansatz(params):
    qc = QuantumCircuit(3, 2)
    qc.compose(bit_flip_encode(), inplace=True)

    # Apply ansatz to logical qubits (all three physical qubits)
    for i in range(len(params)):
        qc.ry(params[i], 0)
        qc.ry(params[i], 1)
        qc.ry(params[i], 2)

    qc.cx(0, 1)
    qc.cx(1, 2)

    qc.compose(syndrome_measure(), inplace=True)
    qc.compose(correction(), inplace=True)

    # Decode
    qc.cx(0, 1)
    qc.cx(0, 2)

    qc.draw('mpl')
    return qc

# Since primitives Estimator does not accept classical registers directly,
# the protected ansatz must be adapted for actual use in practice with custom workflow.

print("\n--- Part 4: Protected VQE ---")
print("Protected ansatz created with 3-qubit bit-flip code and syndrome correction.")

# Visualization
qc_protected = protected_ansatz(init_params)
qc_protected.draw('mpl')

# Analysis
labels = ['Ideal', 'Noisy']
values = [result[1], result_noisy[1]]

plt.bar(labels, values, color=['green', 'red'])
plt.ylabel('Ground State Energy')
plt.title('VQE with and without Noise')
plt.show()

print("\n--- Analysis ---")
print("Protected circuit was constructed. For energy estimation with error correction, further post-processing is typically required.")



